In [ ]:
# 데이터 처리
import numpy as np
import pandas as pd

# 크롤링 모듈
from bs4 import BeautifulSoup
import requests

# 시간처리 모듈
from datetime import datetime
import time

# 텍스트 처리
from konlpy.tag import Twitter

# 한 셀에서 여러 output 한번에 표시
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 크롤링+불용어 처리

In [ ]:
house_list = ["그리핀도르","후플푸프","래번클로","슬리데린"]

list_name = []
list_house = []
list_description = []

for house in house_list :
    
    print("{}의 학생들을 크롤링 합니다".format(house))
    webpage = requests.get('https://namu.wiki/w/' + house)
    soup = BeautifulSoup(webpage.content, "html.parser")
    for a in soup.select("td > div.wiki-paragraph > div > a.wiki-link-internal") :
        list_name.append(a["title"])
        list_house.append(house)
    
    
print("학생들의 description을 크롤링 합니다")
for name in list_name:
    try :
        webpage = requests.get('https://namu.wiki/w/' + name)
        soup = BeautifulSoup(webpage.content, "html.parser")
        name_contents = soup.find('div', {'class':'wiki-heading-content'})
        list_description.append(name_contents.text)
        time.sleep(0.7)
    except:
        list_description.append('no data')
    
print("불용어 처리를 시작합니다")

del_list = ["하다","있다","되다","이다","돼다","않다","그렇다","아니다","이렇다","어떻다","되어다","같다",
            "호그와트","래번클로","후플푸프","슬리데린","그리핀도르",
            '이', '그', '저', '것', '건', '때', '더', '걸', '중', '별', '거',
            "시리즈","등장인물","등장","인물","해리","포터"]

list_cleaned = []    
for i,content in enumerate(list_description) : 
    twitter = Twitter()
    each_description_pos_tagged = twitter.pos(content , norm=True , stem=True)

    globals()["word_cleaned_{}".format(i)] = []
    for word in each_description_pos_tagged :
        if (word[1] in ["Noun","Adjective","Verb"])&(len(word[0]) != 1)&(word[0] not in del_list) :
            globals()["word_cleaned_{}".format(i)].append(word[0])
    
    list_cleaned.append(" ".join(globals()["word_cleaned_{}".format(i)]))

print("불용어 처리 완료!")

# df 생성
df = pd.DataFrame({
    "name" : list_name,
    "house" : list_house,
    "description" : list_cleaned
})
df

# 추가적인 전처리 차근차근

In [ ]:
# index 87번처럼, 불용어 처리 후에 description이 비어있는 학생들이 있다 (개요/소개가 '해리포터 시리즈의 등장인물'로만 되어있던 학생들)
# 래번클로 중 except "no data" 처리된 학생의 description 역시 비어있는 것을 확인할 수 있다
df[df["house"] == "래번클로"]

In [ ]:
df_result = df.copy()

# description이 빈 행의 개수 파악
df_result[df_result['description'] == ""].index

In [ ]:
# description이 비어있지 않은 인덱스만 남기기
df_result = df_result[df_result['description'] != ""].reset_index(drop=True)
df_result

In [ ]:
# df_result.to_excel("data_harrypotter.xlsx")

# (Run)크롤링+불용어 처리+전처리 냅다 모아

In [ ]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests

from datetime import datetime
import time

from konlpy.tag import Twitter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

house_list = ["그리핀도르","후플푸프","래번클로","슬리데린"]

list_name = []
list_house = []
list_description = []

for house in house_list :
    
    print("{}의 학생들을 크롤링 합니다".format(house))
    webpage = requests.get('https://namu.wiki/w/' + house)
    soup = BeautifulSoup(webpage.content, "html.parser")
    for a in soup.select("td > div.wiki-paragraph > div > a.wiki-link-internal") :
        list_name.append(a["title"])
        list_house.append(house)
    
    
print("학생들의 description을 크롤링 합니다")
for name in list_name:
    try :
        webpage = requests.get('https://namu.wiki/w/' + name)
        soup = BeautifulSoup(webpage.content, "html.parser")
        name_contents = soup.find('div', {'class':'wiki-heading-content'})
        list_description.append(name_contents.text)
        time.sleep(0.7)
    except:
        list_description.append('no data')
    
print("불용어 처리를 시작합니다")

del_list = ["하다","있다","되다","이다","돼다","않다","그렇다","아니다","이렇다","어떻다","되어다","같다",
            "호그와트","래번클로","후플푸프","슬리데린","그리핀도르",
            '이', '그', '저', '것', '건', '때', '더', '걸', '중', '별', '거',
            "시리즈","등장인물","등장","인물","해리","포터"]

list_cleaned = []    
for i,content in enumerate(list_description) : 
    twitter = Twitter()
    each_description_pos_tagged = twitter.pos(content , norm=True , stem=True)

    globals()["word_cleaned_{}".format(i)] = []
    for word in each_description_pos_tagged :
        if (word[1] in ["Noun","Adjective","Verb"])&(len(word[0]) != 1)&(word[0] not in del_list) :
            globals()["word_cleaned_{}".format(i)].append(word[0])
    
    list_cleaned.append(" ".join(globals()["word_cleaned_{}".format(i)]))

print("불용어 처리 완료!")

# df 생성
df = pd.DataFrame({
    "name" : list_name,
    "house" : list_house,
    "description" : list_cleaned
})
df

df_result = df.copy()

# description이 비어있지 않은 인덱스만 남기기
df_result = df_result[df_result['description'] != ""].reset_index(drop=True)
df_result

df_result.to_excel("data_harrypotter.xlsx")

# 유사도 분석 차근차근

In [ ]:
유사도 분석
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df_all = df_result.copy()

In [ ]:
# 입력 받고 df_all에 추가하기
name = input('이름을 입력해주세요 : ')
wish = input('해리포터 영화 속으로 들어갈 수 있다면 무엇을 해보고 싶으신가요?! : ')
df_all = df_all.append({'name' : name , 'house' : '머글 기숙사', 'description' : wish},ignore_index=True)

In [ ]:
df_all

In [ ]:
# 각 학생들의 description을 벡터 공간(array)로 표현
vectorizer = TfidfVectorizer()
x_matrix = vectorizer.fit_transform(df_all["description"])
x_matrix.shape

# 유사도(각 학생들을 서로 서로 비교한 결과값) array 생성
cosine_matrix = cosine_similarity(x_matrix, x_matrix)
cosine_matrix.shape

In [ ]:
# index 번호 추출을 위해 시리즈 만들기
for_index = pd.Series(df_all.index,index=df_all["name"])
for_index

In [ ]:
idx = for_index[name] # 내가 입력한 name의 index 번호 
sim_scores = [(i,c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] 
#내가 입력한 wish와 각 학생들의 description 유사도 값 쭈루룩(유사도 값이 1.0인, 내가 입력한 행은 제외)
sim_scores 

In [ ]:
df_all.iloc[61]["name"] # Roger Davies : He was the Captain of the Ravenclaw Quidditch team and played as a Chaser.

In [ ]:
sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
sim_scores[:5] # cosine_similarity 상위 5위까지

In [ ]:
# 이름과 기숙사, 유사도 값을 포함한 dataframe 보여주기
num = [i[0] for i in sim_scores[:5]]
num

simil_df = df_all.iloc[num].copy()
del simil_df["description"]
simil_df["score"] = [i[1] for i in sim_scores[:5]]
simil_df

# (Run)차근차근 ㄴㄴ! 냅다 모아서 함수로 만들어버림!

In [26]:
# df_result = pd.read_excel("data_harrypotter.xlsx",index_col=0)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 이름과 기숙사, 유사도 값을 포함한 dataframe 출력하는 함수
def WingardiumLeviosa(name,wish) :
    df_all = df_result.copy() # 함수 안에서 사용할 dataframe copy
    df_all = df_all.append({'name' : name , 'house' : '머글 기숙사', 'description' : wish},ignore_index=True)

    # 각 학생들의 description을 벡터 공간(array)로 표현
    vectorizer = TfidfVectorizer()
    x_matrix = vectorizer.fit_transform(df_all["description"])

    # 유사도(각 학생들을 서로 서로 비교한 결과값) array 생성
    cosine_matrix = cosine_similarity(x_matrix, x_matrix)

    for_index = pd.Series(df_all.index,index=df_all["name"]) # index 번호 추출을 위해 시리즈 만들기
    idx = for_index[name] # 내가 입력한 name의 index 번호 
    sim_scores = [(i,c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 내가 입력한 wish와 각 학생들의 description 유사도 값 쭈루룩(유사도 값이 1.0인, 내가 입력한 행은 제외)
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True) # 유사도가 높은 순서대로 정렬
    sim_scores[:5] # cosine_similarity 상위 5위까지

    # 이름과 기숙사, 유사도 값을 포함한 dataframe 보여주기
    num = [i[0] for i in sim_scores[:5]] # 상위 5위까지의 index 번호 리스트
    df_simil = df_all.iloc[num].copy() # 상위 5위까지의 dataframe만 가져와 copy
    del df_simil["description"] # 출력에 불필요한 컬럼 지우기
    df_simil["score"] = [i[1] for i in sim_scores[:5]] #유사도값 컬럼 추가하기
    
    return df_simil
    del df_all # 함수 안에서 사용한 dataframe copy본 없애기

# (Run)시현해주세요

In [29]:
# 입력을 받고 주문을 외쳐주세요
name = input('이름을 입력해주세요 : ')
wish = input('해리포터 영화 속으로 들어갈 수 있다면 무엇을 해보고 싶으신가요?! : ')
WingardiumLeviosa(name,wish)

이름을 입력해주세요 : 윤지윤
해리포터 영화 속으로 들어갈 수 있다면 무엇을 해보고 싶으신가요?! : 님부스 2000 타고 퀴디치 경기 뛰기


,name,house,score
61,로저 데이비스,래번클로,0.074881
37,세드릭 디고리,후플푸프,0.073867
19,케이티 벨,그리핀도르,0.066677
13,찰리 위즐리,그리핀도르,0.044415
83,블레이즈 자비니,슬리데린,0.042197
